<a href="https://colab.research.google.com/github/TheMarksan/cc-cg/blob/main/Bresenham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introdução**

A computação gráfica trata-se da síntese de imagens de objetos reais ou imaginários a partir de modelos computacionais. Antes de prosseguir, revisitaremos alguns conceitos introdutórios:

***Síntese de imagens:*** objetos tais como segmentos de reta, polígonos, poliedros, esferas etc.

***Processamento de imagens:*** imagens já prontas para visualização, as quais são transferidas para o computador para serem manipuladas.

***OpenGL***: Sistema gráfico utilizado em modo imediato ou com display list

***Vector graphics***: classe de gráficos para obtenção de figuras a partir de segmentos de reta (vetores)

***Raster graphics (matriciais)***: classe de gráficos para obtenção de figuras a partir do preenchimento de uma matriz de pontos (pixels)

***Resolução gráfica:*** o número de posições endereçáveis (ou pixels) horizontais/verticais que um dispositivo I/O pode distinguir.

***Parâmetros de resol. gráfica:***

ndh (posições horizontalmente): número de unidades lógicas/discretas na horizontal (por ex.: 8 retangulos em 800px = 800/8 = 100)
ndv (posições verticalmente): número de unidades lógicas/discretas na vertical (por ex.: 10 retangulos em 800px = 800/10 = 80)
width (largura do retângulo de visualização em mm) e height (altura do retângulo de visualização em mm)



### **Resolução horizontal:** $$ {horiz\_res} = \frac{ndh}{width} $$

### **Tamanho ponto horizontal:** $$ {horiz\_dot\_res} = \frac{width}{ndh} $$

### **Resolução vertical:** $$ {vert\_res} = \frac{ndv}{heigth} $$

### **Tamanho ponto vertical:** $$ {vert\_dot\_res} = \frac{height}{ndv} $$

### **Total pontos endereçáveis:** $$ {total\_nr\_dots} = {width.}{ndh} $$

### **Resolução de área:** $$ {area\_res} = \frac{total\_nr\_dots}{{width.}{height}} $$

### **Razão de aspecto gráfica:** $$ {aspect\_ratio} = \frac{vert\_dot\_size}{horiz\_dot\_size} $$

### **Razão de aspecto física:** $$ {physical\_aspect\_ratio} = \frac{height}{width} $$


### **Coordenadas (do dispositivo):**

*Horizontalmente:* 0 ≤ dcx (coordenada) ≤ ndh − 1

*Verticalmente:* 0 ≤ dcy (coordenada) ≤ ndv − 1

### **Coordenadas normalizadas do dispositivo - independe de dispositivos gráficos específicos (ndcx, ndcy) :**

*Horizontalmente:* 0 ≤ ndcx ≤ 1

*Verticalmente:* 0 ≤ ndcy ≤ 1

NDC (0,0): origem
NDC (1,1): pixel do canto superior direito ((ndhm1,nhdvm1) no dispositivo)

### **Coordenadas físicas :**

*pcx:* coordenadas fracionária/distância física ao longo do eixo x a partir do extremo esquerdo do retãngulo de visualização - de 0.0 a 1.0

*pcy:* coordenadas fracionária/distância física ao longo do eixo y a partir do extremo inferior - de 0.0 a 1.0

pixel_x = trunc(pcx * width)
pixel_y = trunc(pcy * height)

### **Tranformação de coordenadas físicas p/ coordenadas do dispositivo :**

$$ {dcx} = {trunc(}{ndhm1}\frac{pcx}{width}{)} $$
$$ {dcy} = {trunc(}{ndvm1}\frac{pcy}{height}{)} $$


### **Coordenadas do usuário :**

*xmin ≤ **x** ≤ xmax*

*ymin ≤ **y** ≤ ymax*

### **Transformação de coordenadas do usuário (x, y) para NDCs (ndcx, ndcy):**

*Ps.: área retangular no espaçoo bidimensional*

$$ {ndcx} = \frac{x-xmin}{xmax-xmin} $$

$$ {ndcy} = \frac{y-xmin}{ymax-ymin} $$




In [15]:
# Conversão de coordenadas de entrada (em pixels) para coordenadas normalizadas (NDC)
# Dpi/ppi = pixels per inch
def inp_to_ndc(dcy, dcx, width, height):

  ndcx = (2.0 * dcx) / width - 1.0
  ndcy = -(2.0 * dcy) / height + 1.0

  return ndcx, ndcy

# Default 96
# p (tamanho em cm) to disp (tamanho em pixels)
# 1 inch = 2.54 cm
def p_to_disp(xf_cm, yf_cm, dpi=96):
  dcx = xf_cm * dpi / 2.54
  dcy = yf_cm * dpi / 2.54
  return dcx, dcy

# Inverso de inp_to_ndc
def ndc_to_user(ndc_x, ndc_y, width, height):

    user_x = (ndc_x + 1) * (width / 2)
    user_y = (1 - ndc_y) * (height / 2)
    return user_x, user_y


# Centro de um tela de 15.6 polegadas
xwidth = 17.28
xheight = 9.72
dcx, dcy = p_to_disp(xwidth, xheight, 141.21)

ndcx, ndcy = inp_to_ndc(dcy, dcx, dcx*2, dcy*2)

#Teste inp_to_ndc
print(ndcx)
print(ndcy)


userx, usery = ndc_to_user(ndcx, ndcy, dcx*2, dcy*2)
#Teste ndc_to_user
print(userx)
print(usery)








0.0
0.0
960.672755905512
540.3784251968505


# **Aula 3 - Rasterização**


1. Indicar que localizações serão calculadas pelo algoritmo de Bresenham quando se gera por varredura um segmento de reta entre (1,1) e (8,5) em coordenadas da tela;

2. Escreva um programa que execute o algoritmo de Bresenham e que seja capaz de provar o resultado obtido no exercício 1;